In [1]:
from dataset_loader import DatasetLoader
import json
import pandas as pd
import os
import json
import utils
from tqdm import tqdm
import random
random.seed(42)

In [2]:
merged_train = [json.loads(l) for l in open("../hf_data/merged_train.jsonl")]
merged_dev = [json.loads(l) for l in open("../hf_data/merged_dev.jsonl")]
merged_test = [json.loads(l) for l in open("../hf_data/merged_test.jsonl")]

In [7]:
merged_train[25000]

{'question': 'ek haseena thi ek deewana turkish drama cast',
 'claim': 'Natasha and her fiance, Sunny, visit her ancestral property where they meet Devdhar and form a passionate connection with him.[1]',
 'claim_raw_string': 'Natasha and her fiance, Sunny, visit her ancestral property where they meet Devdhar and form a passionate connection with him.[1]',
 'response': 'Natasha and her fiance, Sunny, visit her ancestral property where they meet Devdhar and form a passionate connection with him.[1]Ek Haseena Thi Ek Deewana Tha (2017) is a romantic drama directed by Suneel Darshan.[2]The movie features a soundtrack of Yaseer Desai, music by Nadeem, mixed by Adam Whittaker, and lyrics by Nadeem.[2]The cast includes Party guest (as Yesmien Bagh Ali), production manager (as Sajida Ahmad), assistant re recording mixer (as Devobrat Chaliha), and sound editor/sound designer (as Aakash Chaudhary).[3]',
 'references': ['[1] She sets off for her destination wedding with fiancé, Sunny, to her ances

In [9]:
def display_ratio(data_list, src_dataset="any"):
    attributable_count = sum(1 for item in data_list if item['attribution_label'] == 'attributable' and (src_dataset == "any" or item["src_dataset"] == src_dataset))
    not_attributable_count = sum(1 for item in data_list if item['attribution_label'] == 'not attributable' and (src_dataset == "any" or item["src_dataset"] == src_dataset))
    total_count = attributable_count + not_attributable_count
    
    attributable_ratio = attributable_count / total_count
    not_attributable_ratio = not_attributable_count / total_count
    ratio_ratio = attributable_ratio / not_attributable_ratio
    
    print(f"Total Count: {total_count}")
    print(f"Attributable Count: {attributable_count} ({attributable_ratio:.2%})")
    print(f"Not Attributable Count: {not_attributable_count} ({not_attributable_ratio:.2%})")
    print(f"Attributable / Not Attributable Ratio: {ratio_ratio:.2f}")
    print("")

display_ratio(merged_train)
display_ratio(merged_train, src_dataset="AttributedQA")
display_ratio(merged_train, src_dataset="HAGRID")
display_ratio(merged_train, src_dataset="ExpertQA")
display_ratio(merged_train, src_dataset="Stanford-GenSearch")

Total Count: 29374
Attributable Count: 19062 (64.89%)
Not Attributable Count: 10312 (35.11%)
Attributable / Not Attributable Ratio: 1.85

Total Count: 8253
Attributable Count: 3053 (36.99%)
Not Attributable Count: 5200 (63.01%)
Attributable / Not Attributable Ratio: 0.59

Total Count: 2977
Attributable Count: 2248 (75.51%)
Not Attributable Count: 729 (24.49%)
Attributable / Not Attributable Ratio: 3.08

Total Count: 8827
Attributable Count: 6092 (69.02%)
Not Attributable Count: 2735 (30.98%)
Attributable / Not Attributable Ratio: 2.23

Total Count: 9317
Attributable Count: 7669 (82.31%)
Not Attributable Count: 1648 (17.69%)
Attributable / Not Attributable Ratio: 4.65



In [12]:
attributedqa_train = [item for item in merged_train if item["src_dataset"] == "AttributedQA"]
attributedqa_dev = [item for item in merged_dev if item["src_dataset"] == "AttributedQA"]
attributedqa_test = [item for item in merged_test if item["src_dataset"] == "AttributedQA"]
hagrid_train = [item for item in merged_train if item["src_dataset"] == "HAGRID"]
hagrid_dev = [item for item in merged_dev if item["src_dataset"] == "HAGRID"]
hagrid_test = [item for item in merged_test if item["src_dataset"] == "HAGRID"]
expertqa_train = [item for item in merged_train if item["src_dataset"] == "ExpertQA"]
expertqa_dev = [item for item in merged_dev if item["src_dataset"] == "ExpertQA"]
expertqa_test = [item for item in merged_test if item["src_dataset"] == "ExpertQA"]
stanford_train = [item for item in merged_train if item["src_dataset"] == "Stanford-GenSearch"]
stanford_dev = [item for item in merged_dev if item["src_dataset"] == "Stanford-GenSearch"]
stanford_test = [item for item in merged_test if item["src_dataset"] == "Stanford-GenSearch"]

In [15]:
import random

def balance_data(data_list, src_dataset="any"):
    # 分离 "attributable" 和 "not attributable" 数据
    attributable_data = [item for item in data_list if item['attribution_label'] == 'attributable']
    not_attributable_data = [item for item in data_list if item['attribution_label'] == 'not attributable']
    
    # 计算两类数据的数量
    min_count = min(len(attributable_data), len(not_attributable_data))
    if src_dataset == "AttributedQA":
        min_count = min(min_count, 1000)
    
    # 随机采样，使得两类数据的数量相等
    balanced_data = random.sample(attributable_data, min_count) + random.sample(not_attributable_data, min_count)
    
    return balanced_data

balanced_attributedqa_train = balance_data(attributedqa_train, src_dataset="AttributedQA")
balanced_hagrid_train = balance_data(hagrid_train)
balanced_expertqa_train = balance_data(expertqa_train)
balanced_stanford_train = balance_data(stanford_train)
balanced_attributed_dev = balance_data(attributedqa_dev)
balanced_hagrid_dev = balance_data(hagrid_dev)
balanced_expertqa_dev = balance_data(expertqa_dev)
balanced_stanford_dev = balance_data(stanford_dev)
balanced_attributed_test = balance_data(attributedqa_test)
balanced_hagrid_test = balance_data(hagrid_test)
balanced_expertqa_test = balance_data(expertqa_test)
balanced_stanford_test = balance_data(stanford_test)
balanced_train = balanced_attributedqa_train + balanced_hagrid_train + balanced_expertqa_train + balanced_stanford_train
balanced_dev = balanced_attributed_dev + balanced_hagrid_dev + balanced_expertqa_dev + balanced_stanford_dev
balanced_test = balanced_attributed_test + balanced_hagrid_test + balanced_expertqa_test + balanced_stanford_test
print("After balancing:")
display_ratio(balanced_attributedqa_train)
display_ratio(balanced_hagrid_train)
display_ratio(balanced_expertqa_train)
display_ratio(balanced_stanford_train)
display_ratio(balanced_attributed_dev)
display_ratio(balanced_hagrid_dev)
display_ratio(balanced_expertqa_dev)
display_ratio(balanced_stanford_dev)
display_ratio(balanced_attributed_test)
display_ratio(balanced_hagrid_test)
display_ratio(balanced_expertqa_test)
display_ratio(balanced_stanford_test)

After balancing:
Total Count: 2000
Attributable Count: 1000 (50.00%)
Not Attributable Count: 1000 (50.00%)
Attributable / Not Attributable Ratio: 1.00

Total Count: 1458
Attributable Count: 729 (50.00%)
Not Attributable Count: 729 (50.00%)
Attributable / Not Attributable Ratio: 1.00

Total Count: 5470
Attributable Count: 2735 (50.00%)
Not Attributable Count: 2735 (50.00%)
Attributable / Not Attributable Ratio: 1.00

Total Count: 3296
Attributable Count: 1648 (50.00%)
Not Attributable Count: 1648 (50.00%)
Attributable / Not Attributable Ratio: 1.00

Total Count: 336
Attributable Count: 168 (50.00%)
Not Attributable Count: 168 (50.00%)
Attributable / Not Attributable Ratio: 1.00

Total Count: 64
Attributable Count: 32 (50.00%)
Not Attributable Count: 32 (50.00%)
Attributable / Not Attributable Ratio: 1.00

Total Count: 334
Attributable Count: 167 (50.00%)
Not Attributable Count: 167 (50.00%)
Attributable / Not Attributable Ratio: 1.00

Total Count: 172
Attributable Count: 86 (50.00%)
Not

In [17]:
with open("../hf_data/train_all_subset_balanced.jsonl", "w") as f:
    for item in balanced_train:
        f.write(json.dumps(item) + "\n")
with open("../hf_data/dev_all_subset_balanced.jsonl", "w") as f:
    for item in balanced_dev:
        f.write(json.dumps(item) + "\n")
with open("../hf_data/test_all_subset_balanced.jsonl", "w") as f:
    for item in balanced_test:
        f.write(json.dumps(item) + "\n")

In [18]:
print(len(balanced_train))
print(len(balanced_dev))
print(len(balanced_test))

12224
906
1876
